In [1]:
# import os
# from PIL import Image
# from pillow_heif import register_heif_opener
# from PIL import ImageFile

# # Cho phép xử lý tệp bị lỗi hoặc không đầy đủ
# ImageFile.LOAD_TRUNCATED_IMAGES = True

# # Đăng ký HEIF/HEIC opener với Pillow
# register_heif_opener()

# def convert_heic_to_png(heic_file, png_file):
#     """Chuyển đổi tệp HEIC sang PNG"""
#     try:
#         image = Image.open(heic_file)  # Mở tệp HEIC
#         image.save(png_file, "PNG")   # Lưu dưới dạng PNG
#         print(f"Converted {heic_file} to {png_file}")
#     except Exception as e:
#         print(f"Failed to convert {heic_file}: {e}")

# # Thư mục chứa tệp HEIC
# file_path = "./DATA"

# # Duyệt qua tất cả các tệp trong thư mục
# for file in os.listdir(file_path):
#     if file.lower().endswith(".heic"):  # Kiểm tra đuôi tệp
#         heic_file = os.path.join(file_path, file)
#         png_file = os.path.splitext(heic_file)[0] + ".png"

#         # Kiểm tra nếu file PNG đã tồn tại
#         if os.path.exists(png_file):
#             print(f"Skipping {file}, {png_file} already exists.")
#             continue

#         # Chuyển đổi nếu file PNG chưa tồn tại
#         convert_heic_to_png(heic_file, png_file)
#     else:
#         print(f"Skipping {file}, not a HEIC file.")

# print("Done")


In [2]:
# !jupyter nbconvert --to script run.ipynb

In [3]:
# !python mmocr_sam.py --inputs ./data/DataFood/IMG_3455.png --outdir ./OUTPUT --device cpu

In [ ]:
import os
import shutil
import cv2
import torch
from mmocr.apis.inferencers import MMOCRInferencer
import warnings
warnings.filterwarnings("ignore")

c:\Users\NGUYEN\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Hàm scale ảnh
def scale_image(image, scale_factor):
    """Scale image by a given factor."""
    height, width = image.shape[:2]
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    scaled_image = cv2.resize(
        image, (new_width, new_height), interpolation=cv2.INTER_LINEAR
    )
    return scaled_image

In [6]:
# Hàm xử lý ảnh OCR
def process_image(image_path, output_dir, mmocr_inferencer, scale_factor):
    """Process a single image using MMOCR and save results."""
    # Đọc ảnh
    image = cv2.imread(image_path)
    if image is None:
        print(f"Cannot read image: {image_path}")
        return

    # Scale ảnh
    scaled_image = scale_image(image, scale_factor)

    # Thực hiện OCR
    results = mmocr_inferencer(scaled_image, save_vis=True, out_dir=output_dir)

    # Lấy kết quả văn bản
    texts = []
    for prediction in results["predictions"]:
        for text in prediction["rec_texts"]:
            texts.append(text)

    # Ghi kết quả văn bản vào file txt
    if texts:
        output_text_file = os.path.join(
            output_dir,
            os.path.basename(image_path)
            .replace(".jpg", ".txt")
            .replace(".png", ".txt"),
        )
        with open(output_text_file, "w", encoding="utf-8") as f:
            f.write("\n".join(texts))
        print(f"Text saved to: {output_text_file}")

In [7]:
# Xóa dữ liệu cũ trong thư mục OUTPUT
def prepare_output_dir(output_dir):
    """Clear old output directory and create a new one."""
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

In [14]:
# Hàm chính để xử lý tất cả ảnh trong thư mục
def process_all_images(input_dir, output_dir, mmocr_inferencer, scale_factor):
    """Process all images in a directory."""
    # Chuẩn bị thư mục output
    prepare_output_dir(output_dir)

    # Danh sách file ảnh
    image_files = [
        os.path.join(input_dir, f)
        for f in os.listdir(input_dir)
        if f.endswith((".jpg", ".png"))
    ]

    for image_file in image_files:
        if not os.path.exists(image_file):
            print(f"File not found: {image_file}")
            continue
        
        # if len(os.listdir(output_dir)) >= 5:
        #     break
        
        process_image(image_file, output_dir, mmocr_inferencer, scale_factor)

    print("Processing complete. Results saved in:", output_dir)

In [16]:
# Cấu hình MMOCR Inferencer
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
mmocr_inferencer = MMOCRInferencer(
    det="mmocr_dev/configs/textdet/dbnetpp/dbnetpp_swinv2_base_w16_in21k.py",
    det_weights="checkpoints/mmocr/db_swin_mix_pretrain.pth",
    rec="mmocr_dev/configs/textrecog/abinet/abinet_20e_st-an_mj.py",
    rec_weights="checkpoints/mmocr/abinet_20e_st-an_mj_20221005_012617-ead8c139.pth",
    device=DEVICE,
)

Loads checkpoint by local backend from path: checkpoints/mmocr/db_swin_mix_pretrain.pth
11/28 17:19:43 - mmengine - INFO - Delete `relative_position_index` and `relative_coords_table` since we always re-init these params according to the `window_size`, which might cause unwanted but unworried warnings when loading checkpoint.
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.stages.0.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.0.blocks.0.attn.w_msa.relative_position_index, backbone.stages.0.blocks.1.attn.w_msa.relative_coords_table, backbone.stages.0.blocks.1.attn.w_msa.relative_position_index, backbone.stages.1.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.1.blocks.0.attn.w_msa.relative_position_index, backbone.stages.1.blocks.1.attn.w_msa.relative_coords_table, backbone.stages.1.blocks.1.attn.w_msa.relative_position_index, backbone.stages.2.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.2.blocks.0

In [17]:
# Cấu hình thư mục và mô hình
INPUT_DIR = "./data/Test"  # Thư mục chứa ảnh đầu vào
OUTPUT_DIR = "./OUTPUT"  # Thư mục lưu kết quả
SCALE_FACTOR = 1.5  # Hệ số scale ảnh


In [18]:
process_all_images(INPUT_DIR, OUTPUT_DIR, mmocr_inferencer, SCALE_FACTOR)

Text saved to: ./OUTPUT\IMG_3464.txt


Text saved to: ./OUTPUT\IMG_3465.txt


Text saved to: ./OUTPUT\IMG_3466.txt


Text saved to: ./OUTPUT\IMG_3467.txt
Processing complete. Results saved in: ./OUTPUT
